In [16]:
import logging
LOG_FILENAME = '/core/logs/backtest.log'
logging.basicConfig(
    format='%(asctime)s %(levelname)-8s %(message)s',
    level=logging.INFO,
    datefmt='%Y-%m-%d %H:%M:%S',filename=LOG_FILENAME)
from cdcqr.backtest.vbt.runsignal import runsignal
from cdcqr.common.utils import parallel_jobs
import numpy as np
from cdcqr.common.utils import LOCAL_DATA_DIR, dict_hash
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")
from cdcqr.common.utils import LOCAL_DATA_DIR
import json
from cdcqr.data.glassnode.features.feature_loaders import produce_feature_BTCSSR
from cdcqr.backtest.vbt.sample_params import param_dict

pctrank = lambda x: pd.Series(x).rank(pct=True).iloc[-1]
from cdcqr.backtest.vbt.runsignal import wrapped_runsignal


In [ ]:
param_dict_raw = param_dict.copy()
param_dict_raw

In [3]:
feature_data_fname = 'gn_BTCSSR'
feature_data_dir = os.path.join(LOCAL_DATA_DIR, 'feature_data', '{}.pickle'.format(feature_data_fname))
df4a = pd.read_pickle(feature_data_dir)

In [4]:
price = pd.read_pickle(os.path.join(LOCAL_DATA_DIR,'BTC-PERP@ftx.pickle'))


In [5]:
qtl_params = [0.995, 0.99, 0.975, 0.96]
lbw_params = [1000, 2000, 3000, 4000]

tp_params = [3, 5, 7, 10] # 
sl_params = [1, 2, 3 , 5] # ts = sl 
n1_params = [1, 10, 100]
n2_params = [101, 200, 500, 1000, 2000]
rsil_params = [10, 20]
rsilag_params = [14]
maf_params = [5, 60]
mas_params = [100, 200]

In [23]:
qtl_params = [0.995, 0.99]
lbw_params = [1000, 2000]

tp_params = [3, 5, ] # 
sl_params = [1, 2,] # ts = sl 
n1_params = [1, 10,]
n2_params = [101, 200]

rsil_params = [10, 20]
rsilag_params = [14]
maf_params = [5, 60]
mas_params = [100, 200]


In [7]:
feature = 'BTCSSR'

In [8]:
counter = 0
backtest_configs = []
for lbw in lbw_params:
    feature_pctrank = '{}_pctrank{}'.format(feature, lbw)
    df4a[feature_pctrank] = df4a[feature].rolling(lbw).apply(pctrank)
    df4b = df4a[df4a.index>='20210101']
    for qtl in qtl_params:
        signal = df4b[feature_pctrank]>qtl # rolling max, keep the singal for a period (1 hour), good for combining with other signals
        signal = signal.reindex(price.index).ffill()
        for tp in tp_params:
            for sl in sl_params:
                print(qtl, lbw, tp, sl)
                for n1 in n1_params:
                    for n2 in n2_params:
                        counter+=1
                        print(counter)
                        for rsil in rsil_params:
                            rsih = 100-rsil
                            for maf in maf_params:
                                for mas in mas_params:
                                    for railag in rsilag_params:
                                        
                                        param_dict = param_dict_raw.copy()
                                        param_dict['qtl'] = qtl
                                        param_dict['lbw'] = lbw
                                        param_dict['signal'] = signal
                                        param_dict['price'] = price
                                        param_dict['tp'] = tp
                                        param_dict['sl'] = sl
                                        param_dict['n1'] = n1
                                        param_dict['n2'] = n1 + n2
                                        param_dict['rsil'] = rsil
                                        param_dict['rsih'] = rsih
                                        param_dict['maf'] = maf
                                        param_dict['mas'] = mas
                                        param_dict['railag'] = railag
                                        
                                        backtest_configs.append(param_dict)

0.995 1000 3 1
1
2
3
4
0.995 1000 3 2
5
6
7
8
0.995 1000 5 1
9
10
11
12
0.995 1000 5 2
13
14
15
16
0.99 1000 3 1
17
18
19
20
0.99 1000 3 2
21
22
23
24
0.99 1000 5 1
25
26
27
28
0.99 1000 5 2
29
30
31
32
0.995 2000 3 1
33
34
35
36
0.995 2000 3 2
37
38
39
40
0.995 2000 5 1
41
42
43
44
0.995 2000 5 2
45
46
47
48
0.99 2000 3 1
49
50
51
52
0.99 2000 3 2
53
54
55
56
0.99 2000 5 1
57
58
59
60
0.99 2000 5 2
61
62
63
64


In [12]:
len(backtest_configs)

512

In [14]:
bt_config = backtest_configs[0]

In [19]:
ret = wrapped_runsignal(bt_config)

In [ ]:
ret